## 深度卷积神经网络（AlexNet）
计算机视觉流程中真正重要的是数据和特征。也就是说，使用较**干净的数据集和较有效的特征**甚至比机器学习模型的选择对图像分类结果的影响更大。

### 学习表示特征
既然特征如此重要，它该如何表示呢？

我们已经提到，在相当长的时间里，特征都是基于各式各样手工设计的函数从数据中提取的。事实上，不少研究者通过提出新的特征提取函数不断改进图像分类结果。这一度为计算机视觉的发展做出了重要贡献。

然而，另一些研究者则持异议。他们认为特征本身也应该由学习得来。他们还相信，为了表征足够复杂的输入，特征本身应该分级表示。持这一想法的研究者相信，多层神经网络可能可以学得数据的多级表征，并逐级表示越来越抽象的概念或模式。以图像分类为例，并回忆5.1节（二维卷积层）中物体边缘检测的例子。在多层神经网络中，图像的第一级的表示可以是在特定的位置和⻆度是否出现边缘；而第二级的表示说不定能够将这些边缘组合出有趣的模式，如花纹；在第三级的表示中，也许上一级的花纹能进一步汇合成对应物体特定部位的模式。这样逐级表示下去，最终，模型能够较容易根据最后一级的表示完成分类任务。需要强调的是，输入的逐级表示由多层模型中的参数决定，而这些参数都是学出来的。

尽管一直有一群执着的研究者不断钻研，试图学习视觉数据的逐级表征，然而很长一段时间里这些野心都未能实现。这其中有诸多因素值得我们一一分析。

**缺失要素一：数据**

包含许多特征的深度模型需要**大量的有标签的数据**才能表现得比其他经典方法更好。限于早期计算机有限的存储和$90$年代有限的研究预算，大部分研究只基于小的公开数据集。例如，不少研究论文基于加州大学欧文分校$（UCI）$提供的若干个公开数据集，其中许多数据集只有几百至几千张图像。这一状况在$2010$年前后兴起的大数据浪潮中得到改善。特别是，$2009$年诞生的ImageNet数据集包含了$1,000$大类物体，每类有多达数千张不同的图像。这一规模是当时其他公开数据集无法与之相提并论的。$ImageNet$数据集同时推动计算机视觉和机器学习研究进入新的阶段，使此前的传统方法不再有优势。

**缺失要素二：硬件**

深度学习对计算资源要求很高。早期的硬件计算能力有限，这使训练较复杂的神经网络变得很困难。然而，**通用GPU的到来改变了这一格局**。很久以来，$GPU$都是为图像处理和计算机游戏设计的，尤其是针对大吞吐量的矩阵和向量乘法从而服务于基本的图形变换。值得庆幸的是，这其中的数学表达与深度网络中的卷积层的表达类似。通用$GPU$这个概念在$2001$年开始兴起，涌现出诸如$OpenCL$和$CUDA$之类的编程框架。这使得$GPU$也在$2010$年前后开始被机器学习社区使用。

### AlexNet
AlexNet与LeNet的设计理念非常相似，但也有显著的区别。

第一，与相对较小的$LeNet$相比，$AlexNet$包含 **8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层**。下面我们来详细描述这些层的设计。

$AlexNet$第一层中的卷积窗口形状是$11\times 11$。因为$ImageNet$中绝大多数图像的高和宽均比$MNIST$图像的高和宽大$10$倍以上，$ImageNet$图像的物体占用更多的像素，所以需要更大的卷积窗口来捕获物体。第二层中的卷积窗口形状减小到$5\times 5$，之后全采用$3\times 3$。此外，第一、第二和第五个卷积层之后都使用了窗口形状为$3\times 3$、步幅为$2$的最大池化层。而且，$AlexNet$使用的卷积通道数也大于$LeNet$中的卷积通道数数十倍。

紧接着最后一个卷积层的是两个输出个数为$4096$的全连接层。这两个巨大的全连接层带来将近$1$ $GB$的模型参数。由于早期显存的限制，最早的$AlexNet$使用双数据流的设计使一个$GPU$只需要处理一半模型。幸运的是，显存在过去几年得到了长足的发展，因此通常我们不再需要这样的特别设计了。

第二，$AlexNet$将$sigmoid$激活函数改成了更加简单的$ReLU$激活函数。一方面，$ReLU$激活函数的**计算更简单**，例如它并没有$sigmoid$激活函数中的求幂运算。另一方面，$ReLU$激活函数在不同的参数初始化方法下使模型更容易训练。这是由于当$sigmoid$激活函数输出极接近$0$或$1$时，这些区域的梯度几乎为$0$，从而造成反向传播无法继续更新部分模型参数；而$ReLU$激活函数在**正区间的梯度恒为1**。因此，若模型参数初始化不当，$sigmoid$函数可能在正区间得到几乎为$0$的梯度，从而令模型无法得到有效训练。

第三，$AlexNet$通过**丢弃法**（参见3.13节）来控制全连接层的模型复杂度。而LeNet并没有使用丢弃法。

第四，$AlexNet$引入了大量的**图像增广，如翻转、裁剪和颜色变化**，从而进一步扩大数据集来缓解过拟合。我们将在后面的9.1节（图像增广）详细介绍这种方法。

下面我们实现稍微简化过的$AlexNet$。

In [1]:
import time
import torch
from torch import nn
import torchvision

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
        )
    
    def forward(self, img):
        feature = self.conv(img)
        return self.fc(feature.view(img.shape[0], -1))

In [2]:
net = AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5)
    (6): Linear(in_features=4096, o

### 读取数据
虽然论文中$AlexNet$使用$ImageNet$数据集，但因为$ImageNet$数据集训练时间较长，我们仍用前面的$Fashion-MNIST$数据集来演示$AlexNet$。读取数据的时候我们额外做了一步将图像高和宽扩大到$AlexNet$使用的图像高和宽$224$。这个可以通过$torchvision.transforms.Resize$实例来实现。也就是说，我们在$ToTensor$实例前使用$Resize$实例，然后使用$Compose$实例来将这两个变换串联以方便调用

In [3]:
def load_data_fashion_mnist(batch_size, resize=None, root='~/DL/Datasets/FashionMNIST'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size = resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)
    
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=True, num_workers=4)
    
    return train_iter, test_iter

batch_size = 128

# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

### 训练
这时候我们可以开始训练$AlexNet$了。相对于$LeNet$，由于图片尺寸变大了而且模型变大了，所以需要更大的显存，也需要更长的训练时间了。

In [4]:
lr, epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs)

training on: cpu
step 1, train_acc: 0.1250
step 2, train_acc: 0.1133
step 3, train_acc: 0.1016
step 4, train_acc: 0.1074
step 5, train_acc: 0.1031
step 6, train_acc: 0.1029
step 7, train_acc: 0.1105
step 8, train_acc: 0.1104
step 9, train_acc: 0.1102
step 10, train_acc: 0.1164
step 11, train_acc: 0.1229
step 12, train_acc: 0.1237
step 13, train_acc: 0.1340
step 14, train_acc: 0.1501
step 15, train_acc: 0.1599
step 16, train_acc: 0.1709
step 17, train_acc: 0.1788
step 18, train_acc: 0.1931
step 19, train_acc: 0.2039
step 20, train_acc: 0.2113
step 21, train_acc: 0.2217
step 22, train_acc: 0.2330
step 23, train_acc: 0.2415
step 24, train_acc: 0.2477
step 25, train_acc: 0.2544
step 26, train_acc: 0.2629
step 27, train_acc: 0.2691
step 28, train_acc: 0.2771
step 29, train_acc: 0.2815
step 30, train_acc: 0.2862
step 31, train_acc: 0.2883
step 32, train_acc: 0.2942
step 33, train_acc: 0.3023
step 34, train_acc: 0.3095
step 35, train_acc: 0.3170
step 36, train_acc: 0.3242
step 37, train_acc: 

step 297, train_acc: 0.7161
step 298, train_acc: 0.7166
step 299, train_acc: 0.7173
step 300, train_acc: 0.7176
step 301, train_acc: 0.7179
step 302, train_acc: 0.7183
step 303, train_acc: 0.7185
step 304, train_acc: 0.7189
step 305, train_acc: 0.7193
step 306, train_acc: 0.7196
step 307, train_acc: 0.7200
step 308, train_acc: 0.7203
step 309, train_acc: 0.7206
step 310, train_acc: 0.7210
step 311, train_acc: 0.7215
step 312, train_acc: 0.7217
step 313, train_acc: 0.7222
step 314, train_acc: 0.7225
step 315, train_acc: 0.7229
step 316, train_acc: 0.7234
step 317, train_acc: 0.7239
step 318, train_acc: 0.7242
step 319, train_acc: 0.7246
step 320, train_acc: 0.7249
step 321, train_acc: 0.7253
step 322, train_acc: 0.7256
step 323, train_acc: 0.7260
step 324, train_acc: 0.7264
step 325, train_acc: 0.7266
step 326, train_acc: 0.7269
step 327, train_acc: 0.7273
step 328, train_acc: 0.7277
step 329, train_acc: 0.7279
step 330, train_acc: 0.7282
step 331, train_acc: 0.7284
step 332, train_acc:

step 122, train_acc: 0.8651
step 123, train_acc: 0.8651
step 124, train_acc: 0.8650
step 125, train_acc: 0.8652
step 126, train_acc: 0.8650
step 127, train_acc: 0.8651
step 128, train_acc: 0.8651
step 129, train_acc: 0.8653
step 130, train_acc: 0.8651
step 131, train_acc: 0.8652
step 132, train_acc: 0.8653
step 133, train_acc: 0.8654
step 134, train_acc: 0.8653
step 135, train_acc: 0.8652
step 136, train_acc: 0.8652
step 137, train_acc: 0.8651
step 138, train_acc: 0.8654
step 139, train_acc: 0.8653
step 140, train_acc: 0.8655
step 141, train_acc: 0.8657
step 142, train_acc: 0.8656
step 143, train_acc: 0.8655
step 144, train_acc: 0.8659
step 145, train_acc: 0.8665
step 146, train_acc: 0.8667
step 147, train_acc: 0.8668
step 148, train_acc: 0.8663
step 149, train_acc: 0.8662
step 150, train_acc: 0.8662
step 151, train_acc: 0.8664
step 152, train_acc: 0.8663
step 153, train_acc: 0.8665
step 154, train_acc: 0.8666
step 155, train_acc: 0.8666
step 156, train_acc: 0.8664
step 157, train_acc:

KeyboardInterrupt: 

+ AlexNet跟LeNet结构类似，但使用了更多的卷积层和更大的参数空间来拟合大规模数据集ImageNet。它是浅层神经网络和深度神经网络的分界线。
+ 虽然看上去AlexNet的实现比LeNet的实现也就多了几行代码而已，但这个观念上的转变和真正优秀实验结果的产生令学术界付出了很多年。